# 02 — Join → Label → Clean → Export (Kepler DR25)

**Goal:** Build the *frozen* tabular dataset for modeling.  
This notebook turns raw Kepler CSVs into three artifacts:
- `data/processed/features/table_v1.parquet`
- `data/processed/labels/labels_v1.csv`
- `data/processed/splits/split_v1.csv`

---

## Inputs (from `data/raw/kepler/`)
- `kepler_stellar_dr25.csv` — DR25 stellar catalog with astrophysical features and detectability/coverage fields (`teff, logg, feh, radius, mass, kepmag, rrmscdpp* (CDPP), dataspan, dutycycle, st_quarters`) plus per-star counts (`nconfp, nkoi, ntce`).
- `kepler_stellar_dr25_supplement.csv` — supplemental (SPWG) refinements for a subset of stars.
- `koi_cumulative.csv` and `koi_dr25.csv` — KOI tables (used to recompute/verify “candidate/confirmed” counts).

---

## What this notebook does

1) **Lock paths to the repo root**  
   Ensures reads/writes go to `data/...` at the repository root (not inside `notebooks/`). 

2) **Load and normalize keys**  
   Reads the four CSVs and coerces `kepid` to integer consistently.  

3) **Derive observing coverage (`nquarters`)**  
   If only `st_quarters` is present, compute `nquarters` by counting 1’s in the bitstring. This lets us filter ambiguous negatives later. 

4) **Overlay Supplemental on DR25 (left-join by `kepid`)**  
   For `teff, logg, feh, radius, mass`, prefer supplemental values when available; otherwise keep DR25. This yields a single master row per star.
5) **Optional KOI aggregation for “lenient” positives**  
   Aggregate KOIs with disposition ∈ {**CANDIDATE, CONFIRMED**} to `nkoi_pos` by `kepid` (source selectable via `USE_KOI_SOURCE = "cumulative" | "dr25" | "none"`). 

6) **Define two label policies (transparent & testable)**
   - **Strict:** `label_strict = (nconfp ≥ 1)`  
   - **Lenient:** `label_lenient = (nconfp ≥ 1) OR (nkoi_pos ≥ 1)`  
   If KOI aggregation is disabled, `nkoi` from DR25 is used instead of `nkoi_pos`.  

7) **Reduce label noise (coverage guardrails)**  
   Keep *negatives* only if the star had adequate coverage:
   - `nquarters ≥ MIN_QUARTERS` (default 8)  
   - `dutycycle ≥ MIN_DUTY` (default 0.50)  
   - `dataspan ≥` lower quartile (configurable via `DATASPAN_Q`)  
   This avoids treating “not sufficiently observed” as “no planet detected.”  

8) **Select/clean modeling features**  
   Columns kept:  
   `teff, logg, feh, radius, mass, kepmag, rrmscdpp03p0, rrmscdpp06p0, rrmscdpp12p0, dataspan, dutycycle, nquarters`.  
   We coerce to numeric, **median-impute** remaining NaNs, and **winsorize** at 0.1%/99.9% to limit extreme tails. Two lightweight interactions are added: `feh_x_teff`, `radius_x_kepmag`. 

9) **Write artifacts**  
   - **Features:** `features/table_v1.parquet` (one row per `kepid`)  
   - **Labels:** `labels/labels_v1.csv` (raw counts + `label_strict`, `label_lenient`)  

10) **Create stratified Train/Val/Test splits**  
    Stratify by *(Teff bin × label_lenient)* to preserve stellar-type mix across splits.  
    Save to `splits/split_v1.csv` with columns: `kepid, split ∈ {train,val,test}`.  

---

## Configuration knobs (top of the notebook)
- `USE_KOI_SOURCE`: `"cumulative"` (default), `"dr25"`, or `"none"`  
- `MIN_QUARTERS`, `MIN_DUTY`, `DATASPAN_Q`: coverage thresholds for filtering ambiguous negatives  
- `RANDOM_SEED`: controls split reproducibility

---

## Why this design?

- **Reproducibility:** a single, auditable place to build the dataset before any modeling.  
- **Fair evaluation:** label policy is explicit; coverage filtering reduces false negatives; splits are stratified.  
- **Flexibility:** KOI source and thresholds are toggles for ablation/sensitivity analysis.

---

## Sanity checks to run after it finishes
- Inspect class balance: `labels_v1.csv` should have rare positives (especially for `label_strict`).  
- Confirm split counts and that each split contains a similar Teff distribution.  
- Spot-check a few rows for realistic ranges (e.g., `dutycycle∈[0,1]`, CDPPs positive).

*Outputs from this notebook are consumed by the preprocessing/EDA and modeling notebooks that follow.*

In [4]:
from pathlib import Path
import pandas as pd

# Find repo root (the folder that contains data/raw/kepler)
ROOT = Path.cwd()
for up in [ROOT, *ROOT.parents]:
    if (up / "data" / "raw" / "kepler").exists():
        ROOT = up
        break
else:
    raise FileNotFoundError("Could not locate repo root with data/raw/kepler")

RAW = ROOT / "data" / "raw" / "kepler"
OUT = ROOT / "data" / "processed"

# Read files
df_star = pd.read_csv(RAW / "kepler_stellar_dr25.csv")
df_sup  = pd.read_csv(RAW / "kepler_stellar_dr25_supplement.csv")
df_kcum = pd.read_csv(RAW / "koi_cumulative.csv")
df_kdr  = pd.read_csv(RAW / "koi_dr25.csv")

# sanity
print("ROOT:", ROOT)
print((RAW/"kepler_stellar_dr25.csv").exists(),
      (RAW/"kepler_stellar_dr25_supplement.csv").exists(),
      (RAW/"koi_cumulative.csv").exists(),
      (RAW/"koi_dr25.csv").exists())

ROOT: /Users/chrisjuarez/CPSC483_ML_Project
True True True True


In [5]:
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
USE_KOI_SOURCE = "cumulative"  # options: "cumulative", "dr25", "none"
RANDOM_SEED = 42

# Coverage filtering thresholds (critical for data quality)
MIN_QUARTERS = 8      # At least 2 years of observation
MIN_DUTY = 0.50       # At least 50% duty cycle
DATASPAN_Q = 0.25     # Use lower quartile as minimum

print(f"Working directory: {Path.cwd()}")
print(f"Configuration: KOI source={USE_KOI_SOURCE}, Min quarters={MIN_QUARTERS}, Min duty={MIN_DUTY}")

Working directory: /Users/chrisjuarez/CPSC483_ML_Project/notebooks
Configuration: KOI source=cumulative, Min quarters=8, Min duty=0.5


In [6]:
df_star = pd.read_csv(RAW / "kepler_stellar_dr25.csv")
df_sup  = pd.read_csv(RAW / "kepler_stellar_dr25_supplement.csv")
df_koi_cum  = pd.read_csv(RAW / "koi_cumulative.csv")
df_koi_dr25 = pd.read_csv(RAW / "koi_dr25.csv")
print(f"Loaded data shapes:")
print(f"  DR25 stellar: {df_star.shape}")
print(f"  Supplement: {df_sup.shape}")
print(f"  KOI cumulative: {df_koi_cum.shape}")
print(f"  KOI DR25: {df_koi_dr25.shape}")

Loaded data shapes:
  DR25 stellar: (200038, 16)
  Supplement: (197096, 6)
  KOI cumulative: (9564, 5)
  KOI DR25: (8054, 5)


In [7]:
def count_ones(s):
    """Count '1's in the quarters string to get number of observed quarters."""
    if pd.isna(s):
        return 0
    # Convert to string and count 1s
    s_str = str(s).strip().strip('"')
    return s_str.count('1')

# Calculate nquarters BEFORE any type conversions
if "st_quarters" in df_star.columns:
    print("Calculating nquarters from st_quarters...")
    df_star["nquarters"] = df_star["st_quarters"].apply(count_ones)
    
    # Verify the calculation worked
    print(f"nquarters statistics:")
    print(f"  Mean: {df_star['nquarters'].mean():.2f}")
    print(f"  Min: {df_star['nquarters'].min()}, Max: {df_star['nquarters'].max()}")
    print(f"  Zero values: {(df_star['nquarters'] == 0).sum()}")
    
    # Show sample to verify
    sample_idx = df_star['nquarters'] > 0
    if sample_idx.any():
        print("\nSample of calculated nquarters:")
        print(df_star.loc[sample_idx.head(5).index, ['kepid', 'st_quarters', 'nquarters']])

Calculating nquarters from st_quarters...
nquarters statistics:
  Mean: 14.07
  Min: 0, Max: 17
  Zero values: 278

Sample of calculated nquarters:
      kepid        st_quarters  nquarters
0  10000785   1111111111111111         16
1  10000797  11111111111111111         17
2  10000800  11111111111111111         17
3  10000823  11111111111111111         17
4  10000827  11111111111111111         17


In [8]:
# Ensure kepid is integer in all dataframes
for df in [df_star, df_sup, df_koi_cum, df_koi_dr25]:
    if "kepid" in df.columns:
        df["kepid"] = pd.to_numeric(df["kepid"], errors="coerce")
        df = df.dropna(subset=["kepid"])
        df["kepid"] = df["kepid"].astype("int64")

print(f"Key normalization complete")

Key normalization complete


In [9]:
# Columns to overlay from supplement
overlay_cols = ["teff", "logg", "feh", "radius", "mass"]

# Merge supplement data
df = df_star.merge(
    df_sup[["kepid"] + overlay_cols], 
    on="kepid", 
    how="left", 
    suffixes=("", "_sup")
)

# Prefer supplement values when available
for col in overlay_cols:
    if f"{col}_sup" in df.columns:
        df[col] = df[f"{col}_sup"].combine_first(df[col])
        df.drop(columns=[f"{col}_sup"], inplace=True)

print(f"Merged dataset shape: {df.shape}")
print(f"Stars with supplement data: {df_sup['kepid'].nunique()}")

Merged dataset shape: (200038, 17)
Stars with supplement data: 197096


In [10]:
def koi_pos_counts(koi_df):
    """Count positive KOI dispositions per star."""
    if koi_df.empty:
        return pd.DataFrame(columns=["kepid", "nkoi_pos"])
    
    # Find disposition column
    disp_col = None
    for col in ["koi_disposition", "disposition", "koi_pdisposition"]:
        if col in koi_df.columns:
            disp_col = col
            break
    
    if disp_col:
        koi = koi_df.copy()
        koi[disp_col] = koi[disp_col].astype(str).str.upper()
        koi = koi[koi[disp_col].isin(["CANDIDATE", "CONFIRMED"])]
        return koi.groupby("kepid").size().rename("nkoi_pos").reset_index()
    else:
        return pd.DataFrame(columns=["kepid", "nkoi_pos"])

# Select KOI source and aggregate
koi_sources = {
    "cumulative": df_koi_cum,
    "dr25": df_koi_dr25,
    "none": pd.DataFrame()
}

koi_used = koi_sources.get(USE_KOI_SOURCE, pd.DataFrame())
koi_counts = koi_pos_counts(koi_used)

# Merge KOI counts
df = df.merge(koi_counts, on="kepid", how="left")
df["nkoi_pos"] = df["nkoi_pos"].fillna(0).astype(int)

print(f"KOI aggregation using source: {USE_KOI_SOURCE}")
print(f"Stars with positive KOIs: {(df['nkoi_pos'] > 0).sum()}")

KOI aggregation using source: cumulative
Stars with positive KOIs: 3611


In [11]:
# Ensure count columns exist
for col in ["nconfp", "nkoi", "ntce"]:
    if col not in df.columns:
        df[col] = 0

# Create two label policies
df["label_strict"] = (df["nconfp"] >= 1).astype(int)
df["label_lenient"] = ((df["nconfp"] >= 1) | (df["nkoi_pos"] >= 1)).astype(int)

# If KOI aggregation disabled, use nkoi from DR25
if USE_KOI_SOURCE == "none":
    df["label_lenient"] = ((df["nconfp"] >= 1) | (df["nkoi"] >= 1)).astype(int)

print(f"Label statistics:")
print(f"  Strict positives: {df['label_strict'].sum()} ({df['label_strict'].mean()*100:.2f}%)")
print(f"  Lenient positives: {df['label_lenient'].sum()} ({df['label_lenient'].mean()*100:.2f}%)")

Label statistics:
  Strict positives: 1982 (0.99%)
  Lenient positives: 3628 (1.81%)


In [12]:
# Calculate dataspan threshold
dataspan_threshold = df["dataspan"].quantile(DATASPAN_Q) if "dataspan" in df.columns else 0

# Define good coverage
good_coverage = (
    (df["nquarters"] >= MIN_QUARTERS) & 
    (df["dutycycle"] >= MIN_DUTY) & 
    (df["dataspan"] >= dataspan_threshold)
)

print(f"Coverage statistics before filtering:")
print(f"  Total stars: {len(df)}")
print(f"  Good coverage: {good_coverage.sum()} ({good_coverage.mean()*100:.1f}%)")
print(f"  Poor coverage: {(~good_coverage).sum()} ({(~good_coverage).mean()*100:.1f}%)")

# CRITICAL: Keep ALL positives, filter only poorly-observed negatives
df_filtered = df[
    (df["label_lenient"] == 1) |  # Keep all positives
    (good_coverage)                # Keep only well-observed negatives
].copy()

print(f"\nAfter coverage filtering:")
print(f"  Total stars: {len(df_filtered)} (removed {len(df) - len(df_filtered)})")
print(f"  Positives kept: {df_filtered['label_lenient'].sum()} (100% retained)")
print(f"  Negatives kept: {(df_filtered['label_lenient'] == 0).sum()}")
print(f"  New positive rate: {df_filtered['label_lenient'].mean()*100:.2f}%")

# Replace df with filtered version
df = df_filtered

Coverage statistics before filtering:
  Total stars: 200038
  Good coverage: 150331 (75.2%)
  Poor coverage: 49707 (24.8%)

After coverage filtering:
  Total stars: 150762 (removed 49276)
  Positives kept: 3628 (100% retained)
  Negatives kept: 147134
  New positive rate: 2.41%


In [13]:
# Basic features to keep
feature_cols = [
    "teff", "logg", "feh", "radius", "mass", "kepmag",
    "rrmscdpp03p0", "rrmscdpp06p0", "rrmscdpp12p0",
    "dataspan", "dutycycle", "nquarters"
]

# Ensure all features exist and clean
for col in feature_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors="coerce")

# Fill missing values with median
for col in feature_cols:
    if col in df.columns and df[col].isna().any():
        median_val = df[col].median()
        df[col] = df[col].fillna(median_val)
        print(f"Filled {col} missing values with median: {median_val:.3f}")

# Winsorize extreme values
for col in feature_cols:
    if col in df.columns:
        lo, hi = df[col].quantile([0.001, 0.999])
        df[col] = df[col].clip(lo, hi)

print(f"Basic features processed: {len(feature_cols)}")

Filled mass missing values with median: 0.987
Filled kepmag missing values with median: 14.481
Filled rrmscdpp03p0 missing values with median: 144.674
Filled rrmscdpp06p0 missing values with median: 117.165
Filled rrmscdpp12p0 missing values with median: 97.240
Filled dataspan missing values with median: 1458.931
Filled dutycycle missing values with median: 0.874
Basic features processed: 12


In [14]:
# Create domain-specific features
print("Creating astrophysics-informed features...")

# 1. Basic interactions (original)
if {"feh", "teff"}.issubset(df.columns):
    df["feh_x_teff"] = df["feh"] * df["teff"]

if {"radius", "kepmag"}.issubset(df.columns):
    df["radius_x_kepmag"] = df["radius"] * df["kepmag"]

# 2. Transit probability indicator
if {"radius", "teff"}.issubset(df.columns):
    df["transit_prob"] = df["radius"] / (df["radius"] + 0.00465 * df["teff"])

# 3. Stellar density proxy
if {"mass", "radius"}.issubset(df.columns):
    df["stellar_density"] = df["mass"] / (df["radius"].clip(lower=0.1) ** 3)

# 4. Noise ratios (detectability)
if {"rrmscdpp06p0", "kepmag"}.issubset(df.columns):
    df["noise_star_ratio"] = df["rrmscdpp06p0"] / df["kepmag"].clip(lower=1)

# 5. Noise consistency (activity proxy)
if {"rrmscdpp03p0", "rrmscdpp12p0"}.issubset(df.columns):
    df["noise_consistency"] = df["rrmscdpp03p0"] / df["rrmscdpp12p0"].clip(lower=1)

# 6. Observation quality score
if {"dutycycle", "nquarters", "dataspan"}.issubset(df.columns):
    df["obs_quality"] = (df["dutycycle"] * df["nquarters"]) / df["dataspan"].clip(lower=1)

# 7. Detection efficiency
if all(col in df.columns for col in ["radius", "kepmag", "rrmscdpp06p0"]):
    df["detection_eff"] = (df["radius"] ** 2) / (
        df["kepmag"].clip(lower=1) * df["rrmscdpp06p0"].clip(lower=1)
    )

# 8. Log transforms for highly skewed CDPP features
for col in ["rrmscdpp03p0", "rrmscdpp06p0", "rrmscdpp12p0"]:
    if col in df.columns:
        df[f"{col}_log"] = np.log1p(df[col])

# Get all feature columns
all_feature_cols = [col for col in df.columns if col not in 
                   ["kepid", "label_strict", "label_lenient", "nconfp", "nkoi", 
                    "ntce", "nkoi_pos", "st_quarters"]]

print(f"Total features created: {len(all_feature_cols)}")
print(f"New features: {[col for col in all_feature_cols if col not in feature_cols]}")

Creating astrophysics-informed features...
Total features created: 23
New features: ['feh_x_teff', 'radius_x_kepmag', 'transit_prob', 'stellar_density', 'noise_star_ratio', 'noise_consistency', 'obs_quality', 'detection_eff', 'rrmscdpp03p0_log', 'rrmscdpp06p0_log', 'rrmscdpp12p0_log']


In [15]:
# Prepare feature and label dataframes
feat_cols = ["kepid"] + all_feature_cols
lab_cols = ["kepid", "nconfp", "nkoi", "ntce", "label_strict", "label_lenient"]

# Save features and labels
df[feat_cols].to_parquet(OUT/"features/table_v1.parquet", index=False)
df[lab_cols].to_csv(OUT/"labels/labels_v1.csv", index=False)

print(f"Saved features: {len(all_feature_cols)} columns")
print(f"Saved labels for {len(df)} stars")

Saved features: 23 columns
Saved labels for 150762 stars


In [16]:
def teff_bin(x):
    """Bin effective temperature for stratification."""
    if pd.isna(x):
        return -1
    elif x < 3800:
        return 0  # M dwarfs
    elif x < 5200:
        return 1  # K dwarfs  
    elif x < 6000:
        return 2  # G dwarfs
    else:
        return 3  # F dwarfs and hotter

# Reset index to ensure continuous indices for splitting
df = df.reset_index(drop=True)

# Prepare stratification
df["teff_bin"] = df["teff"].apply(teff_bin) if "teff" in df.columns else 0
df["strata"] = df["label_lenient"].astype(str) + "_" + df["teff_bin"].astype(str)

# Initialize split column
df["split"] = "train"

# First split: 80% train+val, 20% test
sss1 = StratifiedShuffleSplit(n_splits=1, test_size=0.20, random_state=RANDOM_SEED)
train_val_idx, test_idx = next(sss1.split(df.index, df["strata"]))
df.loc[test_idx, "split"] = "test"

# Second split: From the 80%, take 10% for validation (10/80 = 0.125)
train_val_df = df.loc[train_val_idx]
sss2 = StratifiedShuffleSplit(n_splits=1, test_size=0.125, random_state=RANDOM_SEED+1)
train_idx, val_idx = next(sss2.split(train_val_df.index, train_val_df["strata"]))
df.loc[train_val_df.iloc[val_idx].index, "split"] = "val"

# Save splits (drop temporary columns)
df[["kepid", "split"]].to_csv(OUT/"splits/split_v1.csv", index=False)

# Print split statistics
print("\nSplit statistics:")
for split in ["train", "val", "test"]:
    subset = df[df["split"] == split]
    n_samples = len(subset)
    n_positive = subset["label_lenient"].sum()
    pos_rate = subset["label_lenient"].mean()
    print(f"  {split:5s}: {n_samples:6d} samples, {n_positive:4d} positives ({pos_rate:.2%})")

print(f"\nTotal samples: {len(df)}")
print(f"Overall positive rate: {df['label_lenient'].mean():.2%}")

# Verify stratification worked
rates = df.groupby("split")["label_lenient"].mean()
if rates.std() < 0.001:
    print("✅ Splits are well-balanced")
else:
    print(f"⚠️ Warning: Positive rates vary by {rates.std():.4f}")


Split statistics:
  train: 105532 samples, 2540 positives (2.41%)
  val  :  15077 samples,  363 positives (2.41%)
  test :  30153 samples,  725 positives (2.40%)

Total samples: 150762
Overall positive rate: 2.41%
✅ Splits are well-balanced


In [19]:
print("\n" + "="*60)
print("DATA PREPARATION COMPLETE")
print("="*60)

print(f"""
Dataset Summary:
- Total stars processed: {len(df):,}
- Features created: {len(all_feature_cols)}
- Positive examples: {df['label_lenient'].sum():,} ({df['label_lenient'].mean()*100:.2f}%)
- Coverage filtered negatives: {len(df) - len(df_filtered) if 'df_filtered' in locals() else 0:,}

 Files Created:
- features/table_v1.parquet ({len(all_feature_cols)} features)
- labels/labels_v1.csv (labels and counts)
- splits/split_v1.csv (train/val/test assignments)
""")

# Save processing metadata
metadata = {
    "processing_date": pd.Timestamp.now().isoformat(),
    "n_features": len(all_feature_cols),
    "n_samples": len(df),
    "positive_rate": df['label_lenient'].mean(),
    "coverage_thresholds": {
        "min_quarters": MIN_QUARTERS,
        "min_duty": MIN_DUTY,
        "dataspan_quantile": DATASPAN_Q
    }
}

pd.Series(metadata).to_csv(OUT/"processing_metadata.csv")


DATA PREPARATION COMPLETE

Dataset Summary:
- Total stars processed: 150,762
- Features created: 23
- Positive examples: 3,628 (2.41%)
- Coverage filtered negatives: 0

 Files Created:
- features/table_v1.parquet (23 features)
- labels/labels_v1.csv (labels and counts)
- splits/split_v1.csv (train/val/test assignments)

